# State class
틱택토 상태를 나타낼 수 있는 class를 만들어 주세요.

- 2명의 플레이어가 있고, 각 플레이어의 수가 뭔지 식별가능해야 합니다.  

**꼭 구현해야할 매소드(주니어/시니어 공통)**
- 둘 수 있는 위치가 어딘지 알려주는 함수
- win / lose / draw를 식별해주는 함수
- render 함수

**min-max algorithm / alpha-beta algorithm 구현(시니어)**

### 🚨 만약 감이 안 온다면 운영진에게 연락주시면 참고할 만한 레퍼런스를 보내드리겠습니다.
https://github.com/Jpub/AlphaZero/blob/master/6_7_tictactoe/game.py
### 🚨 참고한 레퍼런스가 있다면, 출처를 남겨주세요.

https://symxsym.tistory.com/entry/pygame-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EA%B2%8C%EC%9E%84-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8B%B1%ED%83%9D%ED%86%A0

https://github.com/kekmodel/gym-tictactoe-zero/blob/master/selfplay_gpu.py render있고 교재에서 나온 코드임

https://medium.com/byte-sized-code/building-a-game-of-tic-tac-toe-in-python-72dd59c5f240
pygame 안 쓰는거

https://m.blog.naver.com/lsszz210/222238307295

In [ ]:
# 패키지 임포트
import random
import math

class State:
    # 초기화
    def __init__(self, pieces=None, enemy_pieces=None):
        # 돌의 배치
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9

    # 돌의 수 얻기
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        return count

    # 승리 여부 판정
    def is_win(self):
        # 돌 3개 연결 여부 확인
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                        self.pieces[x + y * 3] == 0:
                    return False
                x, y = x + dx, y + dy
            return True

        # 승리 여부 판정
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 패배 여부 판정
    def is_lose(self):
        # 돌 3개 연결 여부 확인
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                        self.enemy_pieces[x + y * 3] == 0:
                    return False
                x, y = x + dx, y + dy
            return True

        # 패배 여부 판정
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 무승부 여부 확인
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9

    # 게임 종료 여부 확인
    def is_done(self):
        return self.is_win() or self.is_lose() or self.is_draw()

    # 다음 상태 얻기
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)

    # 합법적인 수의 리스트 얻기
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions

    # 선 수 여부 확인
    def is_first_player(self):
        return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    def render(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        for i in range(9):
            if self.pieces[i] == 1:
                print(ox[0], end='')  # 자신의 돌 표시
            elif self.enemy_pieces[i] == 1:
                print(ox[1], end='')  # 상대의 돌 표시
            else:
                print('-', end='')  # 빈 칸 표시
            if i % 3 == 2:  # 3번째 돌마다 줄바꿈
                print()

In [ ]:
pieces = [1, 0, 0,
          0, 1, 0,
          0, 0, 0]

enemy_pieces = [0, 1, 0,
                0, 0, 1,
                0, 0, 0]
st = State(pieces, enemy_pieces)
st.render()

ox-
-ox
---


In [ ]:
class Minimax:
    def __init__(self, state):
        self.state = state

    def minimax(self, state, max_player):
        if state.is_done():
            if state.is_win():
                return 1 if max_player else -1
            elif state.is_lose():
                return -1 if max_player else 1
            else:
                return 0

        if max_player:
            max_eval = -float('inf')
            for action in state.legal_actions():
                eval = self.minimax(state.next(action), False)
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for action in state.legal_actions():
                eval = self.minimax(state.next(action), True)
                min_eval = min(min_eval, eval)
            return min_eval

    def best_action(self):
        best_score = -float('inf')
        best_move = -1
        for action in self.state.legal_actions():
            score = self.minimax(self.state.next(action), False)
            if score > best_score:
                best_score = score
                best_move = action
        return best_move

In [ ]:
import time
def play_game():
    state = State()  # 게임 초기 상태
    while not state.is_done():
        state.render()
        if state.is_first_player():
            # 사용자의 차례
            print("Your turn! Choose a position (0-8): ")
            action = int(input())
            if action not in state.legal_actions():
                print("잘못된 입력입니다. 다시 선택해주세요")
                continue
        else:
            # 컴퓨터의 차례
            print("Computer's turn...")
            ai = Minimax(state)
            start_time = time.time()
            action = ai.best_action()
            end_time = time.time()
            print(f"Computer took {end_time - start_time:.2f} seconds to decide.")

        # 상태 업데이트
        state = state.next(action)

    # 결과 출력
    state.render()
    if state.is_win():
        print("You win!" if state.is_first_player() else "Computer wins!")
    elif state.is_lose():
        print("Computer wins!" if state.is_first_player() else "You win!")
    else:
        print("It's a draw!")

In [ ]:
play_game()

---
---
---
Your turn! Choose a position (0-8): 
3
---
o--
---
Computer's turn...
Computer took 1.07 seconds to decide.
x--
o--
---
Your turn! Choose a position (0-8): 
4
x--
oo-
---
Computer's turn...
Computer took 0.02 seconds to decide.
x--
oox
---
Your turn! Choose a position (0-8): 
1
xo-
oox
---
Computer's turn...
Computer took 0.00 seconds to decide.
xo-
oox
-x-
Your turn! Choose a position (0-8): 
2
xoo
oox
-x-
Computer's turn...
Computer took 0.00 seconds to decide.
xoo
oox
xx-
Your turn! Choose a position (0-8): 
8
xoo
oox
xxo
It's a draw!
